In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

from pyspark.ml.feature import StringIndexer

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation


In [ ]:
config = SparkConf().setAll([('spark.executor.memory', '8g'), ('spark.executor.cores', '10'), ('spark.cores.max', '12'), ('spark.driver.memory','8g')])
config.setAppName("proj")
config.set("spark.dynamicAllocation.minExecutors", "4");
config.set("spark.dynamicAllocation.maxExecutors", "8");
config.set("spark.dynamicAllocation.initialExecutors", "6"); # the number must be between the min and max
sc = SparkContext(conf=config)  # start a new sc with the current config
spark = SparkSession(sc)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-336631661066794> in <module> 
 4 config . set ( "spark.dynamicAllocation.maxExecutors" , "8" ) ; 
 5 config . set ( "spark.dynamicAllocation.initialExecutors" , "6" ) ; # the number must be between the min and max 
 ----> 6 sc = SparkContext ( conf = config ) # start a new sc with the current config 
 7 spark = SparkSession ( sc ) 

 /databricks/spark/python/pyspark/context.py in __init__ (self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls) 
 132 " is not allowed as it is a security risk.")
 133 
 --> 134 SparkContext . _ensure_initialized ( self , gateway = gateway , conf = conf ) 
 135 try : 
 136 self._do_init(master, appName, sparkHome, pyFiles, environment, batchSize, serializer,

 /databricks/spark/python/pyspark/context.py in _ensure_initialized (cls, instance, gateway, conf) 
 345 " created by %s at %s:%s " 
 346 % (currentAppName, currentMaster,
 --> 347 callsite.function, callsite.file, callsite.linenum))
 348 else : 
 349 SparkContext . _active_spark_context = instance

 ValueError : Cannot run multiple SparkContexts at once; existing SparkContext(app=Databricks Shell, master=local[8]) created by __init__ at /local_disk0/tmp/1606610146529-0/PythonShell.py:1686

In [ ]:
feature_df = spark.read.csv('/FileStore/tables/train_targets_scored.csv', header='true', inferSchema= 'true')   # path in HDFS file system

Code below: transpose dataframe such that it will have the size of num_of_labels x num_of_samples..  
Remember the original target dataframe has the size of num_of_samples x num_of_labels

In [ ]:
vectorized_df_transpose = spark.createDataFrame(feature_df.select(*feature_df.columns[1:]).toPandas().T)  # Tranposed label dataset

In [ ]:
def test_func1(x):

    return None

In [ ]:
_ = vectorized_df_transpose.rdd.map(test_func1).collect() # This runs fine

In [ ]:
_

Out[8]: [None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
from pyspark.ml.classification import LogisticRegression

def test_func2(x):
    rf = LogisticRegression(labelCol="label", featuresCol="features")
#     an_arbitary_pyspark_dataframe_object = spark.createDataFrame([(1,2), (3,4)])
    return None

In [ ]:
_ = vectorized_df_transpose.rdd.map(test_func2).collect()  # This will not run propoerly, meaning that you are unable to run ML algorithm inside map function in Spark

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-336631661066800> in <module> 
 ----> 1 _ = vectorized_df_transpose . rdd . map ( test_func2 ) . collect ( ) 

 /databricks/spark/python/pyspark/rdd.py in collect (self) 
 901 # Default path used in OSS Spark / for non-credential passthrough clusters: 
 902 with SCCallSiteSync ( self . context ) as css : 
 --> 903 sock_info = self . ctx . _jvm . PythonRDD . collectAndServe ( self . _jrdd . rdd ( ) ) 
 904 return list ( _load_from_socket ( sock_info , self . _jrdd_deserializer ) ) 
 905 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 42, ip-10-172-247-11.us-west-2.compute.internal, executor driver): org.apache.spark.api.python.PythonException: 'AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?', from <command-336631661066798>, line 4. Full traceback below:
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 654, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 646, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 279, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/util.py", line 109, in wrapper
 return f(*args, **kwargs)
 File "<command-336631661066798>", line 4, in test_func2
 File "/databricks/spark/python/pyspark/__init__.py", line 110, in wrapper
 return func(self, **kwargs)
 File "/databricks/spark/python/pyspark/ml/classification.py", line 655, in __init__
 "org.apache.spark.ml.classification.LogisticRegression", self.uid)
 File "/databricks/spark/python/pyspark/ml/wrapper.py", line 65, in _new_java_obj
 java_obj = _jvm()
 File "/databricks/spark/python/pyspark/ml/util.py", line 85, in _jvm
 raise AttributeError("Cannot load _jvm from SparkContext. Is SparkContext initialized?")
AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:598)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:733)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:716)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:551)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:10

See also [https://stackoverflow.com/questions/43428297/run-ml-algorithm-inside-map-function-in-spark](https://stackoverflow.com/questions/43428297/run-ml-algorithm-inside-map-function-in-spark)